#Практическая работа 2
Изучить основные типы атак на веб-приложения: SQL-инъекции, XSS, DDoS.


In [ ]:
import pandas as pd #таблицы
import numpy as np #массивы
import random #рандомные числа
import gym # среда для обучения с подкреплением
import torch # фреймворк для машиного обучения
import torch.nn as nn #реализация слоев для нейтронных сетей
import torch.optim as optim #оптимизация параметров нейроной сети

In [ ]:
from tqdm import trange #визуализация прогрессов
from gym import spaces
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from collections import deque

In [ ]:
webattacks = pd.read_csv("Thursday.csv")#загрузка CSV-файлов
ddos = pd.read_csv("Friday.csv")
normal = pd.read_csv("Monday.csv")

Подготовка данных для задачи классификации, связанной с обнаружением атак в сетевом трафике. Он включает в себя назначение меток, объединение данных, создание целевой переменной, очистку данных, нормализацию, разделение на обучающую и тестовую выборки, а также вывод информации о форме выборок и балансе классов

In [ ]:
# Назначаем метки для различных типов атак
webattacks = webattacks.assign(attack_type="WebAttack")  # Метка для веб-атак
ddos = ddos.assign(attack_type="DDoS")                    # Метка для DDoS-атак
normal = normal.assign(attack_type="Normal")              # Метка для нормального трафика

# Объединяем все датафреймы в один
combined_data = pd.concat([webattacks, ddos, normal], axis=0, ignore_index=True)
combined_data.columns = combined_data.columns.str.strip()  # Удаляем лишние пробелы из названий столбцов

# Создаем целевую переменную: 1 - атака, 0 - нормальный трафик
combined_data["is_attack"] = combined_data["Label"].map(lambda lbl: 0 if lbl == "BENIGN" else 1)

# Заменяем бесконечные значения на NaN и удаляем строки с NaN
cleaned_data = combined_data.replace([np.inf, -np.inf], np.nan).dropna()

# Выбираем признаки, исключая целевую переменную и метки
feature_cols = [col for col in cleaned_data.columns if col not in {"Label", "attack_type", "is_attack"}]
X = cleaned_data[feature_cols]  # Признаки
y = cleaned_data["is_attack"]    # Целевая переменная

# Создаем объект для нормализации данных
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)  # Нормализуем признаки

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y,
    test_size=0.2,  # 20% данных отводится для тестирования
    random_state=42,  # Фиксированное значение для воспроизводимости
    stratify=y  # Сохраняем пропорции классов в выборках
)

# Выводим информацию о формах выборок и балансе классов
print("Форма обучающей выборки:", X_train.shape)
print("Форма тестовой выборки:", X_test.shape)
print("Баланс классов (обучение):", np.bincount(y_train))


Форма обучающей выборки: (740338, 78)
Форма тестовой выборки: (185085, 78)
Баланс классов (обучение): [636174 104164]


4) Создаю среду RL: TrafficEnv

In [ ]:
class NetworkTrafficEnvironment(gym.Env):
    def __init__(self, features, labels):
        super(NetworkTrafficEnvironment, self).__init__()
        self.feature_data = features
        self.target_labels = labels
        self.data_position = 0


        self.observation_space = spaces.Box( #определение пространства наблюдений и действий
            low=-np.inf,
            high=np.inf,
            shape=(features.shape[1],),
            dtype=np.float32
        )
        self.action_space = spaces.Discrete(2)  #0 - пропустить, 1 - заблокировать

    def reset(self):
        """Сброс среды к начальному состоянию"""
        self.data_position = 0
        return self.feature_data[self.data_position]

    def step(self, action):
        """Выполнение одного шага в среде"""
        current_label = self.target_labels[self.data_position]


        reward = 1 if action == current_label else -1#вычисление награды


        self.data_position += 1
        episode_completed = self.data_position >= len(self.feature_data)#обновление позиции и проверка завершения эпизода


        next_state = (#получение следующего состояния
            self.feature_data[self.data_position] if not episode_completed
            else np.zeros_like(self.feature_data[0])
        )

        return next_state, reward, episode_completed, {}

    def render(self, mode="human"):
        pass

In [ ]:
# Определяем нейросетевую модель для Q-обучения
class DeepQNetwork(nn.Module):
    def __init__(self, input_size, num_actions):
        super(DeepQNetwork, self).__init__()
        # Определяем слои нейросети
        self.network = nn.Sequential(
            nn.Linear(input_size, 128),  # Первый слой
            nn.ReLU(inplace=True),        # Функция активации ReLU
            nn.Linear(128, 64),           # Второй слой
            nn.ReLU(inplace=True),        # Функция активации ReLU
            nn.Linear(64, num_actions)    # Выходной слой
        )

    def forward(self, x):
        return self.network(x)  # Прямое распространение через сеть

# Настройка параметров
state_size = X_train.shape[1]  # Размерность состояния
num_actions = 2                # Количество возможных действий
discount_factor = 0.99         # Коэффициент дисконтирования
exploration_rate = 1.0         # Начальная вероятность случайного действия
min_exploration_rate = 0.01    # Минимальная вероятность исследования
exploration_decay = 0.995      # Скорость уменьшения exploration rate
learning_rate = 0.001          # Скорость обучения
sample_batch_size = 64         # Размер батча для обучения
experience_buffer = deque(maxlen=10000)  # Буфер воспроизведения опыта

# Инициализация устройства и модели
compute_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Выбор устройства
q_network = DeepQNetwork(state_size, num_actions).to(compute_device)  # Создание модели
network_optimizer = optim.Adam(q_network.parameters(), lr=learning_rate)  # Оптимизатор
criterion = nn.MSELoss()  # Функция потерь

# Функция для выбора действия
def get_action(current_state, exploration_prob):
    if random.random() < exploration_prob:  # Исследование
        return random.randrange(num_actions)

    current_state = torch.FloatTensor(current_state).unsqueeze(0).to(compute_device)  # Преобразование состояния
    with torch.no_grad():  # Отключение градиентов
        action_values = q_network(current_state)  # Получение значений действий
    return action_values.argmax().item()  # Возврат действия с максимальным значением


Обучение агента

EPISODES = 100

BATCH_SIZE = 256

BUFFER_SIZE = 100000

GAMMA = 0.99

EPS_START, EPS_END = 1.0, 0.02

LR = 0.0005

WARMUP_STEPS = 1000

количество эпизодов, размер батча, размер буфера воспроизведения, коэффициент дисконтирования, начальная и конечная вероятности исследования, скорость обучения и количество шагов для разогрева

In [ ]:

class FastDQN(nn.Module):  # Оптимизированная нейросеть (без BatchNorm для инференса)
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),  # Первый слой с 256 нейронами
            nn.LayerNorm(256),          # Замена BatchNorm на LayerNorm
            nn.LeakyReLU(0.1),          # Функция активации LeakyReLU
            nn.Linear(256, 128),        # Второй слой с 128 нейронами
            nn.LayerNorm(128),          # Замена BatchNorm на LayerNorm
            nn.LeakyReLU(0.1),          # Функция активации LeakyReLU
            nn.Linear(128, output_dim)  # Выходной слой с количеством действий
        )

    def forward(self, x):
        return self.net(x)  # Прямое распространение через сеть



EPISODES = 100
BATCH_SIZE = 256
BUFFER_SIZE = 100000
GAMMA = 0.99
EPS_START, EPS_END = 1.0, 0.02
LR = 0.0005
WARMUP_STEPS = 1000


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Инициализация устройства для вычислений
state_dim = X_train.shape[1]  # Размерность состояния
policy_net = FastDQN(state_dim, 2).to(device)  # Создание экземпляра модели и перенос на устройство
target_net = FastDQN(state_dim, 2).to(device)  # Создание целевой сети
target_net.load_state_dict(policy_net.state_dict())  # Копирование весов из policy_net в target_net

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, weight_decay=1e-5)  # Оптимизатор AdamW
loss_fn = nn.SmoothL1Loss()  # Функция потерь (Smooth L1 Loss)
memory = deque(maxlen=BUFFER_SIZE)  # Буфер воспроизведения опыта
global_step = 0  # Глобальный счетчик шагов



def select_action(state, epsilon):  # Функция выбора действия
    global global_step
    if random.random() < epsilon or global_step < WARMUP_STEPS:
        return random.randint(0, 1)  # Случайное действие

    with torch.no_grad():  # Отключение градиентов для вычислений
        policy_net.eval()  # Переключаем модель в режим оценки
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)  # Преобразование состояния в тензор
        action = policy_net(state_tensor).argmax().item()  # Получение действия с максимальным значением
        policy_net.train()  # Возвращаем модель в режим обучения
        return action



for episode in trange(EPISODES, desc="Обучение"):  # Основной цикл обучения
    state = env.reset()  # Сброс среды
    episode_reward = 0  # Награда за эпизод
    epsilon = max(EPS_END, EPS_START * (0.99 ** episode))  # Обновление вероятности исследования

    for step in range(1000):
        global_step += 1
        action = select_action(state, epsilon)  # Выбор действия
        next_state, reward, done, _ = env.step(action)  # Выполнение действия в среде

        priority = abs(reward) + 0.01  # Сохраняем опыт с простым приоритетом
        memory.append((state, action, reward, next_state, done, priority))  # Добавление опыта в буфер
        episode_reward += reward  # Обновление награды за эпизод
        state = next_state  # Переход к следующему состоянию

        if global_step % 4 == 0 and len(memory) >= BATCH_SIZE:  # Обучение на батче
            batch = random.choices(memory, k=BATCH_SIZE)  # Случайный выбор батча из буфера
            states, actions, rewards, next_states, dones, _ = zip(*batch)  # Распаковка батча

            # Конвертация в тензоры
            states = torch.FloatTensor(np.array(states)).to(device)
            actions = torch.LongTensor(np.array(actions)).unsqueeze(1).to(device)
            rewards = torch.FloatTensor(np.array(rewards)).unsqueeze(1).to(device)
            next_states = torch.FloatTensor(np.array(next_states)).to(device)
            dones = torch.BoolTensor(np.array(dones)).unsqueeze(1).to(device)

            with torch.no_grad():  # Double DQN
                next_actions = policy_net(next_states).argmax(1, keepdim=True)  # Получение действий для следующего состояния
                next_q = target_net(next_states).gather(1, next_actions)  # Получение Q-значений для следующего состояния
                targets = rewards + GAMMA * next_q * (~dones)  # Вычисление целевых Q-значений

            current_q = policy_net(states).gather(1, actions)  # Получение текущих Q-значений
            loss = loss_fn(current_q, targets)  # Вычисление потерь

            optimizer.zero_grad()  # Обнуление градиентов
            loss.backward()  # Обратное распространение
            torch.nn.utils.clip_grad_norm_(policy_net.parameters(), 10)  # Обрезка градиентов
            optimizer.step()  # Обновление весов

            if global_step % 100 == 0:  # Мягкое обновление целевой сети
                target_net.load_state_dict(policy_net.state_dict())  # Копирование весов из policy_net в target_net

        if done:  # Если эпизод завершен
            break

    print(f"Эпизод {episode + 1}: Награда={episode_reward:.1f}, ε={epsilon:.3f}")  # Вывод информации об эпизоде


Обучение:   1%|          | 1/100 [00:03<06:08,  3.72s/it]

Эпизод 1: Награда=-36.0, ε=1.000


Обучение:   2%|▏         | 2/100 [00:06<05:24,  3.31s/it]

Эпизод 2: Награда=52.0, ε=0.990


Обучение:   3%|▎         | 3/100 [00:08<04:31,  2.80s/it]

...

Эпизод 98: Награда=588.0, ε=0.377


Обучение:  99%|█████████▉| 99/100 [04:32<00:02,  2.76s/it]

Эпизод 99: Награда=644.0, ε=0.373


Обучение: 100%|██████████| 100/100 [04:37<00:00,  2.77s/it]

Эпизод 100: Награда=620.0, ε=0.370


Резульаты предсказаний

In [ ]:
# Преобразуем тестовые данные и метки в массивы NumPy
test_states = np.asarray(X_test)
test_labels = np.asarray(y_test)

# Инициализируем список для хранения предсказаний
test_predictions = []

# Переводим модель в режим оценки
policy_net.eval()

# Выполняем предсказания на тестовых данных
with torch.no_grad():  # Отключаем вычисление градиентов
    for current_state in test_states:
        # Подготавливаем данные для модели
        state_tensor = torch.FloatTensor(current_state).unsqueeze(0).to(device)  # Преобразуем состояние в тензор

        q_values = policy_net(state_tensor)  # Получаем Q-значения для текущего состояния

        # Выбираем действие с максимальным Q-значением
        predicted_action = torch.argmax(q_values).item()  # Получаем индекс действия с максимальным Q-значением

        # Сохраняем предсказание
        test_predictions.append(predicted_action)  # Добавляем предсказание в список

# Выводим результаты классификации
print("\nРезультаты классификации:")
print(classification_report(
    test_labels,
    test_predictions,
    target_names=["Без атаки", "Атака"],  # Названия классов
    digits=4  # Количество знаков после запятой
))

# Выводим матрицу ошибок
print("\nМатрица ошибок:")
print(confusion_matrix(test_labels, test_predictions))  # Выводим матрицу ошибок



Результаты классификации:
              precision    recall  f1-score   support

   Без атаки     0.9993    0.9960    0.9977    159044
       Атака     0.9762    0.9959    0.9859     26041

    accuracy                         0.9960    185085
   macro avg     0.9878    0.9959    0.9918    185085
weighted avg     0.9961    0.9960    0.9960    185085


Матрица ошибок:
[[158412    632]
 [   108  25933]]
